In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [50]:
all_assets = pd.read_excel("etf_combined.xlsx", index_col="Date", parse_dates=True)
optimal_portfolio = pd.read_excel("../../results/selected_assets_summary.xlsx", usecols=["Ticker", "Weight"]).set_index("Ticker")
benchmark_returns = pd.read_excel("../raw/benchmarks/ftse_balance_prices.xlsx", parse_dates=True).set_index("Date")

In [52]:
# Align ticker & weights
common = all_assets.columns.intersection(optimal_portfolio.index)

missing = set(optimal_portfolio.index) - set(common)
if missing:
    print(f"No return series for: {missing}")

asset_returns = all_assets[common] 
weights = optimal_portfolio.loc[common, "Weight"].values

portfolio_returns = asset_returns.dot(weights)
portfolio_returns = portfolio_returns.pct_change().dropna().rename("Portfolio Returns")


benchmark_returns = benchmark_returns["Last Price"].pct_change().dropna().rename("Benchmark Returns")


KeyError: 'Last Price'